In [38]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm

import random
import numpy as np
import pandas as pd

import datetime as dt
import time

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

import cv2

import keras

from keras.callbacks import TensorBoard

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from mpl_toolkits.axes_grid1 import ImageGrid

from keras.utils.np_utils import to_categorical

from keras.models import Sequential
from keras.models import Model


from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers import Dense, GlobalAveragePooling2D
from keras import optimizers
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau


from keras import backend as K

# from keras.applications import vgg16, resnet50, mobilenet
# from keras.applications.vgg16 import VGG16
# from keras.applications.resnet50 import ResNet50
# from keras.applications import xception
# from keras.applications import inception_v3

from keras.applications.inception_v3 import InceptionV3

from keras.applications.vgg16 import preprocess_input, decode_predictions

from keras.optimizers import SGD




np.random.seed(2)
sns.set(style='white', context='notebook', palette='deep')

In [39]:
def show_pred(preds, Y, val_breed, index, seq, ran):
    leng = len(preds)
    if seq:
        for i in range(index):
            if ran:
                index = random.randint(0, leng) 
            _, imagenet_class_name, prob = decode_predictions(preds, top=1)[index][0]
            plt.title("Original: " + val_breed[Y[index]] + "\nPrediction: " + imagenet_class_name)
            plt.imshow(X_train[index])
            plt.show()
    else:
            _, imagenet_class_name, prob = decode_predictions(preds, top=1)[index][0]
            plt.title("Original: " + val_breed[Y[index]] + "\nPrediction: " + imagenet_class_name)
            plt.imshow(X_train[index])
            plt.show()
        
def accuracy_func(preds, Y, val_breed):
    leng = len(preds)
    count = 0;
    for i in range(leng):
        _, imagenet_class_name, prob = decode_predictions(preds, top=1)[i][0]
        if val_breed[Y[i]] == imagenet_class_name:
            count+=1
    accuracy = (count/leng)*100
    
    print("Accuracy: ", accuracy)
    return accuracy

In [40]:
cache_dir = expanduser(join('~', '.keras'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)

In [41]:
# #Load the VGG model
# vgg_model = vgg16.VGG16(weights='imagenet')
 
# #Load the Inception_V3 model
# inception_model = inception_v3.InceptionV3(weights='imagenet')
 
# #Load the ResNet50 model
# resnet_model = resnet50.ResNet50(weights='imagenet')
 
# #Load the MobileNet model
# mobilenet_model = mobilenet.MobileNet(weights='imagenet')

In [42]:
training_path = 'data/train'
validation_path = 'data/val'
testing_path = 'data/test'
batch_size = 32
target_size=(224, 224)
norm = 255.0
class_mode='categorical'

In [43]:
train_datagen = ImageDataGenerator(
        rescale=1./norm,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./norm)
test_datagen = ImageDataGenerator(rescale=1./norm)

train_generator = train_datagen.flow_from_directory(
        training_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

test_generator = test_datagen.flow_from_directory(
        testing_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [44]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

In [45]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(2, activation='softmax')(x)


In [46]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [47]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True



0 input_2
1 conv2d_95
2 batch_normalization_95
3 activation_95
4 conv2d_96
5 batch_normalization_96
6 activation_96
7 conv2d_97
8 batch_normalization_97
9 activation_97
10 max_pooling2d_5
11 conv2d_98
12 batch_normalization_98
13 activation_98
14 conv2d_99
15 batch_normalization_99
16 activation_99
17 max_pooling2d_6
18 conv2d_103
19 batch_normalization_103
20 activation_103
21 conv2d_101
22 conv2d_104
23 batch_normalization_101
24 batch_normalization_104
25 activation_101
26 activation_104
27 average_pooling2d_10
28 conv2d_100
29 conv2d_102
30 conv2d_105
31 conv2d_106
32 batch_normalization_100
33 batch_normalization_102
34 batch_normalization_105
35 batch_normalization_106
36 activation_100
37 activation_102
38 activation_105
39 activation_106
40 mixed0
41 conv2d_110
42 batch_normalization_110
43 activation_110
44 conv2d_108
45 conv2d_111
46 batch_normalization_108
47 batch_normalization_111
48 activation_108
49 activation_111
50 average_pooling2d_11
51 conv2d_107
52 conv2d_109
53 co

In [48]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate



sgd = optimizers.Adam()
# sgd = optimizers.SGD()
# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# sgd = optimizer=SGD(lr=0.0001, momentum=0.9)

model.compile(sgd, loss='categorical_crossentropy', metrics=["accuracy"])

# model.compile(loss='mean_squared_error', optimizer='sgd')

In [49]:
model_dir = 'outputs/models/'
log_file = "outputs/logs"

model_file = model_dir+"weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"

In [50]:
checkpoint = keras.callbacks.ModelCheckpoint(model_file, monitor='val_acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

In [51]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None)

In [52]:
tensorboard = keras.callbacks.TensorBoard(log_dir=log_file, histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)
tensorboard.set_model(model) 

In [53]:
callbacks_list = [checkpoint, tensorboard, early_stopping]
# callbacks_list = [checkpoint, tensorboard]

In [54]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit_generator(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    verbose=1,
    callbacks=callbacks_list)

Epoch 1/50
163/163 [==============================] - ETA: 41:53 - loss: 0.6450 - acc: 0.68 - ETA: 35:20 - loss: 0.6716 - acc: 0.73 - ETA: 32:18 - loss: 0.8974 - acc: 0.72 - ETA: 30:56 - loss: 0.8141 - acc: 0.78 - ETA: 30:04 - loss: 0.9644 - acc: 0.78 - ETA: 29:14 - loss: 0.9198 - acc: 0.80 - ETA: 28:41 - loss: 0.9031 - acc: 0.80 - ETA: 28:17 - loss: 0.7960 - acc: 0.82 - ETA: 27:51 - loss: 0.7798 - acc: 0.82 - ETA: 27:31 - loss: 0.7110 - acc: 0.84 - ETA: 27:14 - loss: 0.6867 - acc: 0.84 - ETA: 26:56 - loss: 0.6427 - acc: 0.84 - ETA: 26:41 - loss: 0.6011 - acc: 0.85 - ETA: 26:24 - loss: 0.5931 - acc: 0.85 - ETA: 26:09 - loss: 0.5806 - acc: 0.85 - ETA: 25:56 - loss: 0.5570 - acc: 0.86 - ETA: 25:41 - loss: 0.5368 - acc: 0.86 - ETA: 25:30 - loss: 0.5201 - acc: 0.86 - ETA: 25:18 - loss: 0.5197 - acc: 0.86 - ETA: 25:04 - loss: 0.5238 - acc: 0.86 - ETA: 24:52 - loss: 0.5047 - acc: 0.86 - ETA: 24:40 - loss: 0.5028 - acc: 0.86 - ETA: 24:26 - loss: 0.4900 - acc: 0.86 - ETA: 24:14 - loss: 0.4767 

163/163 [==============================] - ETA: 36:13 - loss: 0.1972 - acc: 0.93 - ETA: 34:20 - loss: 0.1319 - acc: 0.95 - ETA: 35:29 - loss: 0.1475 - acc: 0.94 - ETA: 35:01 - loss: 0.1446 - acc: 0.94 - ETA: 35:00 - loss: 0.1386 - acc: 0.95 - ETA: 33:59 - loss: 0.1419 - acc: 0.94 - ETA: 33:11 - loss: 0.1565 - acc: 0.94 - ETA: 32:24 - loss: 0.1587 - acc: 0.94 - ETA: 31:51 - loss: 0.1539 - acc: 0.94 - ETA: 31:24 - loss: 0.1574 - acc: 0.94 - ETA: 30:54 - loss: 0.1589 - acc: 0.93 - ETA: 30:30 - loss: 0.1512 - acc: 0.93 - ETA: 30:10 - loss: 0.1571 - acc: 0.93 - ETA: 29:55 - loss: 0.1507 - acc: 0.93 - ETA: 29:40 - loss: 0.1474 - acc: 0.94 - ETA: 29:20 - loss: 0.1417 - acc: 0.94 - ETA: 28:59 - loss: 0.1391 - acc: 0.94 - ETA: 28:40 - loss: 0.1433 - acc: 0.94 - ETA: 28:25 - loss: 0.1413 - acc: 0.94 - ETA: 28:08 - loss: 0.1393 - acc: 0.94 - ETA: 27:50 - loss: 0.1353 - acc: 0.95 - ETA: 27:32 - loss: 0.1327 - acc: 0.95 - ETA: 27:14 - loss: 0.1413 - acc: 0.94 - ETA: 27:00 - loss: 0.1451 - acc: 0.94

163/163 [==============================] - ETA: 28:36 - loss: 0.0181 - acc: 1.00 - ETA: 27:45 - loss: 0.0143 - acc: 1.00 - ETA: 27:28 - loss: 0.0252 - acc: 1.00 - ETA: 27:05 - loss: 0.0237 - acc: 1.00 - ETA: 26:54 - loss: 0.0936 - acc: 0.97 - ETA: 26:44 - loss: 0.0822 - acc: 0.97 - ETA: 26:29 - loss: 0.0886 - acc: 0.97 - ETA: 26:19 - loss: 0.0790 - acc: 0.98 - ETA: 26:09 - loss: 0.0733 - acc: 0.98 - ETA: 25:55 - loss: 0.0687 - acc: 0.98 - ETA: 25:45 - loss: 0.0814 - acc: 0.98 - ETA: 25:33 - loss: 0.0863 - acc: 0.97 - ETA: 25:23 - loss: 0.0824 - acc: 0.97 - ETA: 25:13 - loss: 0.0775 - acc: 0.97 - ETA: 25:01 - loss: 0.0866 - acc: 0.97 - ETA: 24:52 - loss: 0.0902 - acc: 0.97 - ETA: 24:43 - loss: 0.0861 - acc: 0.97 - ETA: 24:31 - loss: 0.0889 - acc: 0.97 - ETA: 24:22 - loss: 0.0858 - acc: 0.97 - ETA: 24:13 - loss: 0.0835 - acc: 0.97 - ETA: 24:02 - loss: 0.0806 - acc: 0.97 - ETA: 23:53 - loss: 0.0790 - acc: 0.97 - ETA: 23:41 - loss: 0.0779 - acc: 0.97 - ETA: 23:31 - loss: 0.0822 - acc: 0.97

163/163 [==============================] - ETA: 26:46 - loss: 0.2817 - acc: 0.90 - ETA: 27:04 - loss: 0.1868 - acc: 0.92 - ETA: 27:01 - loss: 0.1377 - acc: 0.93 - ETA: 26:47 - loss: 0.1068 - acc: 0.95 - ETA: 26:41 - loss: 0.0886 - acc: 0.96 - ETA: 26:27 - loss: 0.0769 - acc: 0.96 - ETA: 26:18 - loss: 0.0813 - acc: 0.96 - ETA: 26:11 - loss: 0.0777 - acc: 0.96 - ETA: 25:57 - loss: 0.0724 - acc: 0.96 - ETA: 25:49 - loss: 0.0731 - acc: 0.96 - ETA: 25:40 - loss: 0.0727 - acc: 0.96 - ETA: 25:27 - loss: 0.0687 - acc: 0.96 - ETA: 25:18 - loss: 0.0645 - acc: 0.97 - ETA: 25:09 - loss: 0.0629 - acc: 0.97 - ETA: 24:57 - loss: 0.0600 - acc: 0.97 - ETA: 24:48 - loss: 0.0715 - acc: 0.97 - ETA: 24:36 - loss: 0.0723 - acc: 0.97 - ETA: 24:27 - loss: 0.0758 - acc: 0.97 - ETA: 24:17 - loss: 0.0784 - acc: 0.96 - ETA: 24:06 - loss: 0.0749 - acc: 0.97 - ETA: 23:57 - loss: 0.0731 - acc: 0.97 - ETA: 23:47 - loss: 0.0703 - acc: 0.97 - ETA: 23:36 - loss: 0.0728 - acc: 0.97 - ETA: 23:27 - loss: 0.0782 - acc: 0.97

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [28]:
filepath = 'outputs/models/weights-improvement-02-0.81.hdf5'
model = keras.models.load_model(filepath)

In [ ]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit_generator(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    verbose=1,
    callbacks=callbacks_list)

In [35]:
filepath = 'outputs/models/weights-improvement-02-0.81.hdf5'
model = keras.models.load_model(filepath)

In [55]:
preds = model.predict_generator(test_generator, verbose=1)

20/20 [==============================] - ETA: 13:0 - ETA: 7:5 - ETA: 6: - ETA: 5: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 53s - ETA: 39 - ETA: 26 - ETA: 13 - 255s 13s/step


In [56]:
categorical_accuracy = keras.metrics.categorical_accuracy(test_generator.classes, preds)

In [59]:
result = model.evaluate_generator(generator=test_generator, verbose=1)

20/20 [==============================] - ETA: 4: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 50s - ETA: 34 - ETA: 17 - 336s 17s/step


In [60]:
print("%s%.2f%s"% ("Accuracy: ", result[1]*100, "%"))
print("%s%.2f"% ("Loss: ", result[0]))

Accuracy: 68.11%
Loss: 1.29


In [61]:
def get_class_values(gen):
    vals = []
    for i in gen:
        vals.append(gen[i])
    return vals

def get_classes(gen):
    rev_gen = {}
    for i in gen:
        rev_gen[gen[i]] = i
    return rev_gen

def find_max(preds_i):
    num = -1
    index = -1
    for i in range(len(preds_i)):
        if preds_i[i]>num:
            num = preds_i[i]
            index = i
    return index  

def get_label(cls, val):
    return cls[val]

In [62]:
vals = get_class_values(test_generator.class_indices)
cls = get_classes(test_generator.class_indices)

In [ ]:
total = 0
acc = 0
direc = "data\\pre\\test"
files = test_generator.filenames
for i in range(len(preds)):
    cls_index = find_max(preds[i])
    pred_label = cls[cls_index]
    org_label = cls[test_generator.classes[i]]
    file_name = os.path.join(direc, files[i])
    if org_label==pred_label:
        acc +=1
    if total%10==0:
        result = "Wrong!!!\n"
        if org_label==pred_label:
            result = "Right!!!\n"
        title_text = "Original: "+ org_label+"\nPredicted: "+ pred_label+"\nFile: "+ file_name
        title_text=result+ title_text
        
        img = cv2.imread(file_name, 3)
        if org_label==pred_label:
            plt.title(title_text, color='blue')
        else:
            plt.title(title_text, color='red')
        plt.imshow(img)
        plt.show()
    
    total+=1
accuracy = (acc/total)*100
print("Accuracy: ", accuracy)